In [7]:
import pandas as pd
import json
df_raw = pd.read_pickle('../data/samples.pkl')
df_bands = df_raw.loc[:, ['band', 'album', 'song']].drop_duplicates()

dict_result = (
    df_bands.groupby('band')
    .apply(lambda x: x.groupby('album')['song'].apply(list).to_dict())
    .to_dict()
)

json_result = json.dumps(dict_result)

# with open('band_album_song_data.json', 'w') as file:
#     file.write(json_result)


In [31]:
import json
with open('band_album_song_data.json', 'r') as file:
    json_data = json.load(file)

In [59]:
data = {'Circle of Dust': {'Brainchild': ['Enshrined', 'Course of Ruin', 'Pale Reflection']}}
band = 'Circle of Dust'
album = 'Brainchild'
song = 'Enshrined'

In [82]:
find_band('Age of Chance')


In [8]:
from urllib.parse import unquote
import discogs_client as DiscogsClient
DISCOGS_USER_TOKEN='TFVOBtHcTkDzNyzLtYgqdJteXoXSlKyCjVtnjDNW'
SPOTIFY_CLIENT_ID='6f4d5e1238de4803bca434bb32da9ad5'
SPOTIFY_CLIENT_SECRET='37c32729befa4188b715d6c30135c9e4'
YTMUSIC_ACCESS_TOKEN='ya29.a0AfB_byB9CWG5iwkYBMZeaHiGHguWqsXF2vy3aYdugq1dPKsFT05fWp54cp97fuKATy_fQbe7qoPIxSSMsKxVAM8fbWzdteGNY-8m_J7gMC_df-H6YSYVCiMktQbpQv-7qH53f0zfxdZzm4g8vMElIJ9h5ly4t0MegtuUtKxl1znLkCEqaCgYKAdYSARISFQHGX2Mi-A93ch6xphVdmKGeueBcLw0183'
YTMUSIC_REFRESH_TOKEN='1//09nhOL0gpiD83CgYIARAAGAkSNwF-L9IrfVltzz8nXey8j3x_ZWo6aNcHXa4oQ5S3XAMX28F7qJaCdHo4lV6apuR8pDSf6CCEcpE'

discogs_client = DiscogsClient.Client('ExampleApplication/0.1', user_token=DISCOGS_USER_TOKEN)

def get_artist_discogs(band):
    band = unquote(band)
    result = discogs_client.search(band, type='artist')[0]
    result_dict = {
        'discogs_id': result.id,
        'discogs_name': result.name,
        'discogs_img': result.images[0]['uri'] if result.images else '',
        'discogs_url': result.url,
        'discogs_releases': result.releases,
    }
    return result_dict

def find_band(band_name):
    import re

    band_results = discogs_client.search(band_name, type='artist')
    band_names = [result.name for result in band_results]
    pattern = re.compile(fr'{band_name} \(\d\)')
    multiple = sum(map(lambda x: 1 if re.findall(pattern, x) else 0, band_names)) > 1
    if band_names.count(band_name) == 1 and not multiple:
        return band_results[band_names.index(band_name)]
    band_names_lower = [result.name.lower() for result in band_results]
    if band_names_lower.count(band_name.lower()) == 1 and not multiple:
        return band_results[band_names_lower.index(band_name.lower())]
    return None


df_bands['discogs_id'] = ''

i = 0
for band in df_bands['band'].drop_duplicates():
    try:
        band_obj = find_band(band)
    except Exception:
        continue
    band_id = band_obj.id if band_obj else None
    df_bands.loc[df_bands['band'] == band, 'discogs_id'] = str(band_id)
    print(i, band, band_obj, band_id)
    i+=1

display(df_bands)


0 Age of Chance <Artist 24172 'Age Of Chance'> 24172
1 Amgod <Artist 69877 'amGod'> 69877
2 Aslan Faction <Artist 79541 'Aslan Faction'> 79541
3 ATD Convention <Artist 181295 'ATD Convention'> 181295
4 Bassomatic <Artist 10357 'Bassomatic'> 10357
5 Battery None None
6 Bomb The Bass <Artist 4079 'Bomb The Bass'> 4079
7 Brainchild None None
8 Callisto None None
9 Cat Rapes Dog <Artist 8947 'Cat Rapes Dog'> 8947
10 Cenobita <Artist 135364 'Cenobita'> 135364
11 Circle of Dust <Artist 101966 'Circle Of Dust'> 101966
12 Click Click <Artist 32630 'Click Click'> 32630
13 Covenant None None
14 Crackhead None None
15 Death Ride 69 <Artist 49877 'Death Ride 69'> 49877
16 Deconstruction None None
17 Delerium None None
18 Dillinja <Artist 315 'Dillinja'> 315
19 DJ Icey <Artist 2830 'DJ Icey'> 2830
20 Dom & Roland <Artist 289 'Dom & Roland'> 289
21 Doran None None
22 Dual-Conscience None None
23 Electric Hellfire Club, The None None
24 EMF None None
25 Evil's Toy None None
26 Executor None None
27 F

KeyboardInterrupt: 

In [223]:
i = 0
for band in df_bands.loc[(df_bands['discogs_id'] == '') & (df_bands['band'] != 'Paris') & (df_bands['band'] != 'Was not Was') & (df_bands['band'] != 'H') & (df_bands['band'] != 'Nas') & (df_bands['band'] != 'U2') & (df_bands['band'] != 'Boss')  & (df_bands['band'] != '+killthemanbecomethemonster+'), 'band'].drop_duplicates():
    band_obj = find_band(band)
    band_id = band_obj.id if band_obj else None
    df_bands.loc[df_bands['band'] == band, 'discogs_id'] = str(band_id)
    print(i, band, band_obj, band_id)
    i+=1


0 R.E.M. None None
1 DJ Gizmo None None
2 Bowling Green, The None None
3 Rammstein <Artist 11771 'Rammstein'> 11771
4 Smashing Pumpkins None None


In [222]:
df_bands.loc[(df_bands['discogs_id'] == '') & (df_bands['band'] != 'Paris') & ((df_bands['band'] != 'Was not Was')) & ((df_bands['band'] != 'H')) & ((df_bands['band'] != 'Nas')) & ((df_bands['band'] != 'U2')) & ((df_bands['band'] != 'Boss'))  & ((df_bands['band'] != '+killthemanbecomethemonster+')), 'band'].drop_duplicates()

7721                            Boss
8225    +killthemanbecomethemonster+
8226                          R.E.M.
8227                        DJ Gizmo
8230              Bowling Green, The
8261                       Rammstein
8263               Smashing Pumpkins
Name: band, dtype: object

In [4]:
df_bands.loc[~df_bands['band'].isin(['Paris', 'Was not Was', 'H', 'Nas', 'U2', 'Boss', '+killthemanbecomethemonster+']), 'band']

0                           Age of Chance
1                                   Amgod
6                           Aslan Faction
8                          ATD Convention
11                             Bassomatic
                      ...                
8278                  Front Line Assembly
8279    My Life with the Thrill Kill Kult
8280    My Life with the Thrill Kill Kult
8281                 In Slaughter Natives
8282                         Skinny Puppy
Name: band, Length: 3295, dtype: object

In [13]:
df_bands.loc[(df_bands['discogs_id'] == '') | (df_bands['discogs_id'] == 'None')].head()


,band,album,song,discogs_id
12,Battery,Mutate,Sinister,None
14,Brainchild,Mindwarp,Pale Reflection,None
16,Callisto,"Netherworld, The (EP)",Bounce,None
36,Covenant,Dreams of a Cryotank,Edge of Dawn,None
37,Crackhead,Poo Gazer volume 1: Cat Scat Fever,Alpha State Sales Pitch,None


In [14]:
find_band('Battery')